In [0]:
!pip install scikit-cmeans


In [0]:
!pip install scikit-fuzzy

# INBUILT FUNCTION

In [0]:
import numpy as np
import sklearn as sk
import sklearn.datasets as ds
import skfuzzy as fuzz


In [5]:
iris = ds.load_iris()

labels = iris.target_names
target = iris.target
iris = np.array(iris.data).T
print(labels)
print(target)

clusters = 3
cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(iris, clusters, 2, error=0.005, maxiter=1000, init=None)
iris = iris.T

['setosa' 'versicolor' 'virginica']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [6]:
cntr

array([[5.88824985, 2.76082332, 4.36295471, 1.39680135],
       [5.00396437, 3.4141226 , 1.48276058, 0.25352167],
       [6.77418675, 3.05214199, 5.64575359, 2.05314922]])

In [0]:
def distance(point,centers):
    a=np.sqrt(np.square(point[0]-centers[0][0])+np.square(point[1]-centers[0][1])+np.square(point[2]-centers[0][2])+np.square(point[3]-centers[0][3]))
    b=np.sqrt(np.square(point[0]-centers[1][0])+np.square(point[1]-centers[1][1])+np.square(point[2]-centers[1][2])+np.square(point[3]-centers[1][3]))
    c=np.sqrt(np.square(point[0]-centers[2][0])+np.square(point[1]-centers[2][1])+np.square(point[2]-centers[2][2])+np.square(point[3]-centers[2][3])) 
    return a,b,c

In [8]:
predicted_class=[]
for i in range (iris.shape[0]):
    a,b,c=distance(iris[i],cntr)
    clas=min(a,b,c)
    if(clas==a):
        predicted_class.append(1)
    elif(clas==b):
        predicted_class.append(2)
    elif(clas==c):
        predicted_class.append(0)

import numpy as np
from sklearn.metrics import accuracy_score
accuracy_userdefine = accuracy_score(predicted_class, target)
print('Accuracy of Inbuilt Function is : ',accuracy_userdefine)

Accuracy of Inbuilt Function is :  0.31333333333333335


# USER DEFINE FUNCTION

In [0]:
import pandas as pd
import numpy as np
import random
import operator
import math

#!pip install pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

from matplotlib import pyplot as plt
import pandas as pd
from copy import deepcopy
import numpy as np

def GoogleDriveAuth(ID):
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    file = drive.CreateFile({'id': ID})
    return file
File_ID = '1I0ZKCNu1Y7lF_3is4e39rtsSDSCf9eRW'
file_name = GoogleDriveAuth(File_ID)
file_name.GetContentFile('iris.csv')
df_full = pd.read_csv('iris.csv')


In [0]:
columns = list(df_full.columns)
features = columns[:len(columns)-1]
class_labels = list(df_full[columns[-1]])
df = df_full[features]

# Number of Attributes
num_attr = len(df.columns) - 1

# Number of Clusters
k = 3

# Maximum number of iterations
MAX_ITER = 100

# Number of data points
n = len(df)

# Fuzzy parameter
m = 3.00

In [0]:
def initializeMembershipMatrix():
    membership_mat = list()
    for i in range(n):
        random_num_list = [random.random() for i in range(k)]
        summation = sum(random_num_list)
        temp_list = [x/summation for x in random_num_list]
        membership_mat.append(temp_list)
    return membership_mat

def calculateClusterCenter(membership_mat):
    cluster_mem_val = list(zip(*membership_mat))
    cluster_centers = list()
    for j in range(k):
        x = list(cluster_mem_val[j])
        xraised = [e ** m for e in x]
        denominator = sum(xraised)
        temp_num = list()
        for i in range(n):
            data_point = list(df.iloc[i])
            prod = [xraised[i] * val for val in data_point]
            temp_num.append(prod)
        numerator = map(sum, zip(*temp_num))
        center = [z/denominator for z in numerator]
        cluster_centers.append(center)
    return cluster_centers

def updateMembershipValue(membership_mat, cluster_centers):
    p = float(2/(m-1))
    for i in range(n):
        x = list(df.iloc[i])
        distances = [np.linalg.norm(list(map(operator.sub, x, cluster_centers[j]))) for j in range(k)]
        for j in range(k):
            den = sum([math.pow(float(distances[j]/distances[c]), p) for c in range(k)])
            membership_mat[i][j] = float(1/den)       
    return membership_mat

def getClusters(membership_mat):
    cluster_labels = list()
    for i in range(n):
        max_val, idx = max((val, idx) for (idx, val) in enumerate(membership_mat[i]))
        cluster_labels.append(idx)
    return cluster_labels

def fuzzyCMeansClustering():
    membership_mat = initializeMembershipMatrix()
    curr = 0
    while curr <= MAX_ITER:
        cluster_centers = calculateClusterCenter(membership_mat)
        membership_mat = updateMembershipValue(membership_mat, cluster_centers)
        cluster_labels = getClusters(membership_mat)
        curr += 1
    #print(membership_mat)
    return cluster_labels, cluster_centers
labels, centers = fuzzyCMeansClustering()

In [22]:
df_full.loc[df_full['Species']=='Iris-setosa','actual_label'] =0
df_full.loc[df_full['Species']=='Iris-versicolor','actual_label'] =2
df_full.loc[df_full['Species']=='Iris-virginica','actual_label'] =1

from sklearn.metrics import accuracy_score
print('accuracy is of User Define Function :',end='')
print(accuracy_score(df_full[['actual_label']], labels))

accuracy is of User Define Function :0.9866666666666667
